In [8]:
import sys
sys.path.insert(0, '..')
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import json
import sys
import pandas as pd
import re

from config import reddit

In [9]:
from dataclasses import dataclass

@dataclass
class YoutubeLink:
    url: str
    subreddit: str
    thread_url: str
    
@dataclass
class Comment:
    comment_text: str
    subreddit: str
    thread_url: str

In [10]:
SUBREDDITS = reddit.SUBREDDITS
YoutubeLinks = []
Comments = []

driver = webdriver.Firefox()

In [11]:
def process_thread(url, subreddit):
    # old.reddit.com is easier to scrape, doesn't have js lazy loading
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    
    #Find all links in comments, keeping only youtube links
    comments = soup.find_all('div', {"data-type": "comment"})
    for comment in comments:
        #Record comment text
        comment_text = comment.find('div', {'class': 'md'}).text
        Comments.append(Comment(comment_text, subreddit, url))
        
        #Record youtube links
        for link in comment.find('div', {'class': 'usertext-body'}).find_all('a', href=True):
            if 'youtube.com' in link['href']:
                YoutubeLinks.append(YoutubeLink(link['href'], subreddit, url))
                print(link)


In [12]:
for SUBREDDIT in SUBREDDITS[:1]:
    url = f"https://reddit.com/r/{SUBREDDIT}/top/?sort=top&t=year"
    driver.get(url)
    
    # Get all thread links
    soup = BeautifulSoup(driver.page_source)
    links = soup.find_all('a', {"data-click-id": "body"})
    threads = [link for link in links if link['href'].startswith('/r/')]
    
    for thread in threads[:5]:
        thread_url = f"https://old.reddit.com{thread['href']}"
        links = process_thread(thread_url, SUBREDDIT)
                
driver.close()

<a href="https://m.youtube.com/watch?v=Akm7ik-H_7U">mandatory parking minimums</a>


In [13]:
pd.DataFrame(YoutubeLinks).to_csv('../data/raw/reddit/scraped_youtube_links.csv', index=False)
pd.DataFrame(Comments).to_csv('../data/raw/reddit/scraped_comments.csv', index=False)